In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("krkopt.data", header=None)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28056 entries, 0 to 28055
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       28056 non-null  object
 1   1       28056 non-null  int64 
 2   2       28056 non-null  object
 3   3       28056 non-null  int64 
 4   4       28056 non-null  object
 5   5       28056 non-null  int64 
 6   6       28056 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.5+ MB


In [4]:
data.columns = ["wkc", "wkr", "wrc", "wrr", "bkc", "bkr", "opt rank" ]

In [5]:
# data[["wkc", "wrc", "bkc", "opt rank"]] = data[["wkc", "wrc", "bkc", "opt rank"]].astype('string')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28056 entries, 0 to 28055
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   wkc       28056 non-null  object
 1   wkr       28056 non-null  int64 
 2   wrc       28056 non-null  object
 3   wrr       28056 non-null  int64 
 4   bkc       28056 non-null  object
 5   bkr       28056 non-null  int64 
 6   opt rank  28056 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.5+ MB


In [7]:
# X = data.iloc[:, 0:6]
# y = data['opt rank']

In [8]:
# X

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

In [10]:
train.shape, test.shape

((22444, 7), (5612, 7))

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.5.0'

In [13]:
import tensorflow_decision_forests as tfdf

In [14]:
np.version.version

'1.19.5'

In [15]:
train = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="opt rank")
test = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="opt rank")

In [16]:
train

<BatchDataset shapes: ({wkc: (None,), wkr: (None,), wrc: (None,), wrr: (None,), bkc: (None,), bkr: (None,)}, (None,)), types: ({wkc: tf.string, wkr: tf.int64, wrc: tf.string, wrr: tf.int64, bkc: tf.string, bkr: tf.int64}, tf.int64)>

In [17]:
model = tfdf.keras.RandomForestModel()

In [18]:
model.fit(train)

351/351 [==============================] - 3s 989us/step


In [19]:
model.summary()

Model: "random_forest_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (6):
	bkc
	bkr
	wkc
	wkr
	wrc
	wrr

No weights

Variable Importance: NUM_NODES:
    1. "wrr" 217805.000000 ################
    2. "wrc" 164809.000000 ###########
    3. "bkr" 118635.000000 #######
    4. "bkc" 111789.000000 #######
    5. "wkc" 50842.000000 ##
    6. "wkr" 21767.000000 

Variable Importance: NUM_AS_ROOT:
    1. "bkr" 197.000000 ################
    2. "wkr" 85.000000 ######
    3. "wkc" 15.000000 
    4. "bkc"  3.000000 

Variable Importance: SUM_SCORE:
    1. "wrr" 2951741.765141 ################
    2. "wrc" 2797621.897747 ##############
    3. "bkr" 2761827.488663 ##############
    4. "bkc" 2522625.358567 ##

In [20]:
model.compile(metrics=["accuracy"])
print(model.evaluate(test))


88/88 [==============================] - 1s 11ms/step - loss: 0.0000e+00 - accuracy: 0.7641
[0.0, 0.7640769481658936]


In [21]:
pred

NameError: name 'pred' is not defined

In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0)

In [ ]:
# List all the other available learning algorithms
# tfdf.keras.get_all_models()

# ? tfdf.keras.GradientBoostedTreesModel


# Create another model with specified hyper-parameters
model = tfdf.keras.GradientBoostedTreesModel(
    num_trees=500,
    growing_strategy="BEST_FIRST_GLOBAL",
    max_depth=8,
    split_axis="SPARSE_OBLIQUE"
    )
model.fit(train)



In [ ]:
# Evaluate the model
model.compile(metrics=["accuracy"])
print(model.evaluate(test))

In [ ]:
tfdf.keras.GradientBoostedTreesModel?